# 4D change analysis of near-continuous LiDAR time series for applications in geomorphic monitoring

## Time series-based change analysis of surface dynamics using 4D-OBCs

In this notebook we will perform time series-based surface change analysis on a time series of permanent TLS point clouds of the sandy beach at Kijkduin for a timespan of around 6 months (<a href="#references">Vos et al., 2022</a>). An introduction to the case study and dataset can be found [here](https://3dgeo-heidelberg.github.io/etrainee/module3/06_casestudy_sandybeach/06_casestudy_sandybeach.html). 

The objective is to assess surface dynamics with 4D objects-by-change (following <a href="#references">Anders et al., 2021</a>). Look into the related article for comparison of possible surface dynamics of the use case and help for deciding on suitable parameters, etc.

The workflow is  introduced throughout this notebook. You can also make use of the [software documentation](https://py4dgeo.readthedocs.io/en/latest/pythonapi.html)!

## Software and data

This task is solved using Python with the [`py4dgeo`](https://github.com/3dgeo-heidelberg/py4dgeo) library. 

You can use CloudCompare or GIS Software (e.g. QGIS) to check the data and visualize your results.

The dataset is a subsampled version of the original time series, using 12-hourly epochs of point clouds and spatial subsampling to 50 cm. The dataset [can be downloaded](https://zenodo.org/records/10003575) (module3.zip) from the E-learning course E-TRAINEE. In the data directory `kijkduin`, you find the prepared input point clouds and a core points point cloud, which is manually cleaned from noise.

[E-TRAINEE](https://github.com/3dgeo-heidelberg/etrainee) is an e-learning course on Time Series Analysis in Remote Sensing for Understanding Human-Environment Interactions. This course has been developed by research groups from four partner universities – Charles University, Heidelberg University, University of Innsbruck, and University of Warsaw.


## Loading data and calculation of surface changes

Prepare the analysis by compiling the list of files (epochs) and read the timestamps from the file names (format `YYMMDD_hhmmss`) into `datetime` objects. Use the point cloud files and timestamps to create a py4dgeo `SpatiotemporalAnalysis` object. For this you need to instantiate the M3C2 algorithm. You can use the point cloud file `170115_150816_aoi_50cm.laz` as core points. Explore the point cloud properties in CloudCompare: 

* Considering the available point density and surface characteristics, what would be a suitable cylinder radius for the distance calculation?
* What would be a suitable approach to derive the surface normals in this topography and expected types of surface changes?

Hint: In this flat topography and predominant provess of sand deposition and erosion, it can be suitable to orient the normals purely vertically. In this case, they do not need to be computed, and you can [customize the py4dgeo algorithm](https://py4dgeo.readthedocs.io/en/latest/customization.html#Changing-search-directions) accordingly.

Use the first point cloud in the time series (list of files) as reference epoch. You can assume a registration error of 1.9 cm for the M3C2 distance calculation (cf. <a href="#references">Vos et al., 2022</a>).

Explore the spatiotemporal change information by visualizing the changes at a selected epoch and visualizing the time series at a selected location. 

First, we start by setting up the Python environment and data:

In [1]:
# import required modules
import py4dgeo
import os
import numpy as np
from datetime import datetime
import pooch

# Download data from zenodo and set path to point cloud folder
p = pooch.Pooch(base_url="doi:10.5281/zenodo.10003574/", path=pooch.os_cache("py4dgeo"))
p.load_registry_from_doi()
p.fetch("module3.zip", processor=pooch.Unzip())
pc_dir = os.path.join(p.path, "module3.zip.unzip/module3/kijkduin", "pointclouds")


# list of point clouds (time series)
pc_list = os.listdir(pc_dir)
pc_list[:5]  # print the first elements

['kijkduin_170117_120041.laz',
 'kijkduin_170118_000050.laz',
 'kijkduin_170120_120036.laz',
 'kijkduin_170121_000046.laz',
 'kijkduin_170121_120055.laz']

In the list of point cloud files you can see that we have one laz file per epoch available. The file name contains the timestamp of the epoch, respectively, in format `YYMMDD_hhmmss`. To use this information for our analysis, we read the timestamp information from the file names into `datetime` objects.

In [2]:
# read the timestamps from file names
timestamps = []
for f in pc_list:
    if not f.endswith(".laz"):
        continue

    # get the timestamp from the file name
    timestamp_str = "_".join(f.split(".")[0].split("_")[1:])  # yields YYMMDD_hhmmss

    # convert string to datetime object
    timestamp = datetime.strptime(timestamp_str, "%y%m%d_%H%M%S")
    timestamps.append(timestamp)

timestamps[:5]

[datetime.datetime(2017, 1, 17, 12, 0, 41),
 datetime.datetime(2017, 1, 18, 0, 0, 50),
 datetime.datetime(2017, 1, 20, 12, 0, 36),
 datetime.datetime(2017, 1, 21, 0, 0, 46),
 datetime.datetime(2017, 1, 21, 12, 0, 55)]

Now we use the point cloud files and timestamp information to create a `SpatiotemporalAnalysis` object.

In [3]:
data_path = os.path.join(p.path, "module3.zip.unzip/module3\kijkduin")
print(data_path)
analysis = py4dgeo.SpatiotemporalAnalysis(f"{data_path}/kijkduin.zip", force=True)

C:\Users\ep413\AppData\Local\py4dgeo\py4dgeo\Cache\module3.zip.unzip/module3\kijkduin
[2023-12-18 13:16:17][INFO] Creating analysis file C:\Users\ep413\AppData\Local\py4dgeo\py4dgeo\Cache\module3.zip.unzip/module3\kijkduin/kijkduin.zip


As reference epoch, we use the first epoch in our time series:

In [ ]:
# specify the reference epoch
reference_epoch_file = os.path.join(pc_dir, pc_list[0])

# read the reference epoch and set the timestamp
reference_epoch = py4dgeo.read_from_las(reference_epoch_file)
reference_epoch.timestamp = timestamps[0]

# set the reference epoch in the spatiotemporal analysis object
analysis.reference_epoch = reference_epoch

For epochs to be added, we now configure the M3C2 algorithm to derive the change values. We would like to set the normals purely vertically, so we define a customized computation of cylinder `directions`:

In [ ]:
# Inherit from the M3C2 algorithm class to define a custom direction algorithm
class M3C2_Vertical(py4dgeo.M3C2):
    def directions(self):
        return np.array([0, 0, 1])  # vertical vector orientation


# specify corepoints, here all points of the reference epoch
analysis.corepoints = reference_epoch.cloud[::]

# specify M3C2 parameters for our custom algorithm class
analysis.m3c2 = M3C2_Vertical(
    cyl_radii=(1.0,), max_distance=10.0, registration_error=0.019
)

Now we add all the other epochs with their timestamps:

In [ ]:
# create a list to collect epoch objects
epochs = []
for e, pc_file in enumerate(pc_list[1:]):
    epoch_file = os.path.join(pc_dir, pc_file)
    epoch = py4dgeo.read_from_las(epoch_file)
    epoch.timestamp = timestamps[e]
    epochs.append(epoch)

# add epoch objects to the spatiotemporal analysis object
analysis.add_epochs(*epochs)

# print the spatiotemporal analysis data for 3 corepoints and 5 epochs, respectively
print(f"Space-time distance array:\n{analysis.distances[:3,:5]}")
print(
    f"Uncertainties of M3C2 distance calculation:\n{analysis.uncertainties['lodetection'][:3, :5]}"
)
print(f"Timestamp deltas of analysis:\n{analysis.timedeltas[:5]}")

We visualize the changes in the scene for a selected epoch, together with the time series of surface changes at a selected location. The location here was selected separately in CloudCompare (as the corepoint id).

In [ ]:
cp_idx_sel = 15162  # selected core point index
epoch_idx_sel = 28  # selected epoch index

# import plotting module
import matplotlib.pyplot as plt

# allow interactive rotation in notebook
%matplotlib inline

# create the figure
fig = plt.figure(figsize=(15, 5))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

# get the corepoints
corepoints = analysis.corepoints.cloud

# get change values of last epoch for all corepoints
distances = analysis.distances
distances_epoch = [d[epoch_idx_sel] for d in distances]

# get the time series of changes at a specific core point locations
coord_sel = analysis.corepoints.cloud[cp_idx_sel]
timeseries_sel = distances[cp_idx_sel]

# get the list of timestamps from the reference epoch timestamp and timedeltas
timestamps = [t + analysis.reference_epoch.timestamp for t in analysis.timedeltas]

# plot the scene
d = ax1.scatter(
    corepoints[:, 0],
    corepoints[:, 1],
    c=distances_epoch[:],
    cmap="seismic_r",
    vmin=-1.5,
    vmax=1.5,
    s=1,
    zorder=1,
)
plt.colorbar(d, format=("%.2f"), label="Distance [m]", ax=ax1, pad=0.15)

# add the location of the selected coordinate
ax1.scatter(
    coord_sel[0],
    coord_sel[1],
    facecolor="yellow",
    edgecolor="black",
    s=100,
    zorder=2,
    label="Selected location",
    marker="*",
)
ax1.legend()

# configure the plot layout
ax1.set_xlabel("X [m]")
ax1.set_ylabel("Y [m]")
ax1.set_aspect("equal")
ax1.set_title(
    "Changes at %s"
    % (analysis.reference_epoch.timestamp + analysis.timedeltas[epoch_idx_sel])
)

# plot the time series
ax2.scatter(timestamps, timeseries_sel, s=5, color="black", label="Raw")
ax2.plot(timestamps, timeseries_sel, color="blue", label="Smoothed")
ax2.set_xlabel("Date")

# add the epoch of the plotted scene
ax2.scatter(
    timestamps[epoch_idx_sel],
    timeseries_sel[epoch_idx_sel],
    facecolor="yellow",
    marker="*",
    edgecolor="black",
    s=100,
    color="red",
    label="Selected epoch",
)
ax2.legend()

# format the date labels
import matplotlib.dates as mdates

dtFmt = mdates.DateFormatter("%b-%d")
plt.gca().xaxis.set_major_formatter(dtFmt)

# configure the plot layout
ax2.set_ylabel("Distance [m]")
ax2.grid()
ax2.set_ylim(-0.3, 1.6)
ax2.set_title("Time series at selected location")

plt.tight_layout()
plt.show()

The map of changes in the scene shows us linear structures of sand deposition near the coast which can be interpreted as sand bars (with knowledge about coastal processes). This is confirmed by the surface behavior over time, expressed in the time series plot. However, the time series is quite noisy especially in this part of the beach, which is regularly covered by water during high tides (leading to missing data) and also varies strongly in surface moisture (influencing the LiDAR range measurement and causing noise). We therefore continue with temporal smoothing of the time series.

## Temporal smoothing

We are dealing with a temporal subset of the original hourly time series. The effect of temporal measurement variability may therefore be less pronounced (compared to the assessment in, e.g., [Anders et al., 2019](#references)). Nonetheless, we may apply temporal smoothing to reduce the influence of noise on your change analysis using a rolling median averaging of one week. This will also fill possible gaps in your data, e.g., lower ranges during poor atmospheric conditions or no data due to water coverage during tides on the lower beach part. 

We can visualize the raw and smoothed change values in the time series of your selected location.

We apply a rolling median with a defined temporal window of 14 epochs (corresponding to one week of 12-hourly point clouds) using the `temporal_averaging()` function in py4dgeo.

In [ ]:
analysis.smoothed_distances = py4dgeo.temporal_averaging(
    analysis.distances, smoothing_window=14
)

Now we can compare the raw and smoothed time series at our selected location:

In [ ]:
# create the figure
fig, ax = plt.subplots(1, 1, figsize=(7, 5))

# plot the raw time series
ax.scatter(timestamps, timeseries_sel, color="blue", label="raw", s=5)

# plot the smoothed time series
timeseries_sel_smooth = analysis.smoothed_distances[cp_idx_sel]
ax.plot(timestamps, timeseries_sel_smooth, color="red", label="smooth")

# format the date labels
import matplotlib.dates as mdates

dtFmt = mdates.DateFormatter("%b-%d")
plt.gca().xaxis.set_major_formatter(dtFmt)

# add plot elements
ax.set_xlabel("Date")
ax.set_ylabel("Distance [m]")
ax.grid()
ax.set_ylim(-0.25, 1.25)

plt.tight_layout()
plt.show()

From the smoothed time series at the selected location, we can now much better interpret the surface behavior. In fact, we can distinctly observe that there were two consecutive occurrences of temporary deposition of several weeks. These represent two phases where sand bars are present. They can be extracted as individual objects by the 4D objects-by-change method. Before, we continue with time series clustering and the assessment of overall change patterns in the following.

## Extraction of 4D objects-by-change

Now we use the 4D objects-by-change (4D-OBC) method to identify individual surface activities in the beach scene. The objective is to extract temporary occurrences of accumulation or erosion, as occurs when a sand bar is formed during some timespan, or when sand is deposited by anthropogenic works. This type of surface activity is implemented with the original seed detection in py4dgeo, so you do not need to customize the algorithm. Decide for suitable parameters following <a href="#references">Anders et al. (2021)</a> - but bear in mind that we are using a different temporal resolution, so you may need to adjust the temporal change detection.

We perform the extraction for selected seed locations, e.g. considering interesting clusters of change patterns identified in the previous step. In principle, the spatiotemporal segmentation can also be applied to the full dataset (all time series at all core point locations are used as potential seed candidates), but long processing time needs to be expected.

In this solution, we will use the selected location from above to extract the sand bars as 4D object-by-change. The strength of the method is that the occurrences are identified individually, even though they have spatial overlap, as they can be separated in the time series information.

We instantiate the `RegionGrowingAlgorithm` class using the parameters of [Anders et al, 2021](#references), and run the object extraction:

In [ ]:
# parametrize the 4D-OBC extraction
algo = py4dgeo.RegionGrowingAlgorithm(
    window_width=14,
    minperiod=2,
    height_threshold=0.05,
    neighborhood_radius=1.0,
    min_segments=10,
    thresholds=[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    seed_candidates=[cp_idx_sel],
)

# run the algorithm
analysis.invalidate_results(
    seeds=True, objects=True, smoothed_distances=False
)  # only required if you want to re-run the algorithm
objects = algo.run(analysis)

<span style="color:blue"></span> 

In [ ]:
seed_timeseries = analysis.smoothed_distances[cp_idx_sel]
plt.plot(
    timestamps,
    seed_timeseries,
    c="black",
    linestyle="--",
    linewidth=0.5,
    label="Seed timeseries",
)

for sid, example_seed in enumerate(analysis.seeds):
    seed_end = example_seed.end_epoch
    seed_start = example_seed.start_epoch
    seed_cp_idx = example_seed.index

    plt.plot(
        timestamps[seed_start : seed_end + 1],
        seed_timeseries[seed_start : seed_end + 1],
        label=f"Seed {sid}",
    )

# format the date labels
dtFmt = mdates.DateFormatter("%b-%d")
plt.gca().xaxis.set_major_formatter(dtFmt)

# add plot elements
plt.xlabel("Date")
plt.ylabel("Distance [m]")

plt.legend()
plt.show()

At the selected location, three separate surface activities are detected as seed for 4D-OBC extraction. We may not be satisfied with the determination of the timing. In a real analysis, we would now aim to improve the temporal detection - either by using a different approach of seed detection (cf. [algorithm customization](https://py4dgeo.readthedocs.io/en/latest/4dobc-customization.html#Seed-point-detection)), or by postprocessing the temporal segments from the current seed detection.

Here, we use the result and look into the spatial object properties:

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(15, 7))
ax1, ax2 = axs

# get indices of 4D-OBC
sel_seed_idx = 1
idxs = objects[sel_seed_idx].indices

# get change values at end of object for each location
epoch_of_interest = int(
    (objects[sel_seed_idx].end_epoch - objects[sel_seed_idx].start_epoch) / 2
    + objects[sel_seed_idx].start_epoch
)
distances_of_interest = analysis.smoothed_distances[:, epoch_of_interest]

# get the change magnitude between end and start of object for each location
magnitudes_of_interest = (
    analysis.smoothed_distances[:, epoch_of_interest]
    - analysis.smoothed_distances[:, int(objects[sel_seed_idx].start_epoch)]
)

# set the colormap according to magnitude at each location in the object
crange = 1.0
import matplotlib.colors as mcolors

cmap = plt.get_cmap("seismic_r").copy()
norm = mcolors.CenteredNorm(halfrange=crange)
cmapvals = norm(magnitudes_of_interest)

# plot the timeseries of the segmented locations (colored by time series similarity)
for idx in idxs[::10]:
    ax1.plot(
        timestamps,
        analysis.smoothed_distances[idx],
        c=cmap(cmapvals[idx]),
        linewidth=0.5,
    )

# plot the seed time series
ax1.plot(
    timestamps,
    analysis.smoothed_distances[cp_idx_sel],
    c="black",
    linewidth=1.0,
    label="Seed timeseries",
)

# fill the area of the object
ax1.axvspan(
    timestamps[objects[sel_seed_idx].start_epoch],
    timestamps[objects[sel_seed_idx].end_epoch],
    alpha=0.3,
    color="grey",
    label="4D-OBC timespan",
)

# add legend and format the date labels
dtFmt = mdates.DateFormatter("%b-%d")
plt.gca().xaxis.set_major_formatter(dtFmt)
ax1.legend()

# get subset of core points incorporated in 4D-OBC
cloud = analysis.corepoints.cloud
subset_cloud = cloud[idxs, :2]

# plot coordinates colored by change values at end magnitude of object
d = ax2.scatter(
    cloud[:, 0],
    cloud[:, 1],
    c=magnitudes_of_interest,
    cmap="seismic_r",
    vmin=-crange,
    vmax=crange,
    s=1,
)
plt.colorbar(d, format=("%.2f"), label="Change magnitude [m]", ax=ax2)
ax2.set_aspect("equal")

# plot convex hull of 4D-OBC
from scipy.spatial import ConvexHull
from matplotlib.patches import Polygon

hull = ConvexHull(subset_cloud)
ax2.add_patch(
    Polygon(subset_cloud[hull.vertices, 0:2], label="4D-OBC hull", fill=False)
)

# plot seed location of 4D-OBC
ax2.scatter(
    cloud[cp_idx_sel, 0], cloud[cp_idx_sel, 1], marker="*", c="black", label="Seed"
)

# add plot elements
ax1.set_title("Time series of segmented 4D-OBC locations")
ax1.set_xlabel("Date")
ax1.set_ylabel("Distance [m]")
ax2.set_title(
    f"Magnitudes of change in the 4D-OBC timespan\n({timestamps[epoch_of_interest]-timestamps[analysis.objects[sel_seed_idx].start_epoch]} hours)"
)
ax2.set_xlabel("X [m]")
ax2.set_ylabel("Y [m]")
ax2.legend(loc="upper right")

plt.tight_layout()
plt.show()

You may now check the different spatial extents for the three objects extracted from this seed location. In subsequent analysis, the spatial-temporal extent of each object can be used to describe individual activities (e.g., their change rates, magnitudes, ...) and relating them to one another in their timing and spatial distribution.

<a id='references'></a>
# References

* Anders, K., Lindenbergh, R. C., Vos, S. E., Mara, H., de Vries, S., & Höfle, B. (2019). High-Frequency 3D Geomorphic Observation Using Hourly Terrestrial Laser Scanning Data Of A Sandy Beach. ISPRS Ann. Photogramm. Remote Sens. Spatial Inf. Sci., IV-2/W5, pp. 317-324. doi: [10.5194/isprs-annals-IV-2-W5-317-2019](https://doi.org/10.5194/isprs-annals-IV-2-W5-317-2019).
* Anders, K., Winiwarter, L., Mara, H., Lindenbergh, R., Vos, S. E., & Höfle, B. (2021). Fully automatic spatiotemporal segmentation of 3D LiDAR time series for the extraction of natural surface changes. ISPRS Journal of Photogrammetry and Remote Sensing, 173, pp. 297-308. doi: [10.1016/j.isprsjprs.2021.01.015](https://doi.org/10.1016/j.isprsjprs.2021.01.015).
* Vos, S., Anders, K., Kuschnerus, M., Lindenberg, R., Höfle, B., Aarnikhof, S. & Vries, S. (2022). A high-resolution 4D terrestrial laser scan dataset of the Kijkduin beach-dune system, The Netherlands.  Scientific Data, 9:191. doi: [10.1038/s41597-022-01291-9](https://doi.org/10.1038/s41597-022-01291-9).